<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Transformers in a nutshell
  </div> 
  
<div style="
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Large Language Model
  </div> 


  <div style="
      font-size: 15px; 
      line-height: 12px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Jean-baptiste AUJOGUE
  </div> 


  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
 July 2023
  </div>

<a id="TOC"></a>

#### Table Of Content

1. [Dataset](#data) <br>
2. [Albert finetuning](#albert) <br>
3. [Inference](#inference) <br>



#### Reference

- Hugginface full list of [tutorial notebooks](https://github.com/huggingface/transformers/tree/main/notebooks) (see also [here](https://huggingface.co/docs/transformers/main/notebooks#pytorch-examples))
- Huggingface [tutorial notebook](https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb) on token classification
- Huggingface [course](https://huggingface.co/course/chapter7/2?fw=tf) on token classification
- Huggingface full list of [training scripts](https://github.com/huggingface/transformers/tree/main/examples/pytorch)
- Huggingface [training script](https://github.com/huggingface/transformers/blob/main/examples/pytorch/token-classification/run_ner.py) on token classification
- Huggingface & Pytorch 2.0 [post](https://www.philschmid.de/getting-started-pytorch-2-0-transformers)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

# DL
import torch
import transformers
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# viz
from IPython.display import HTML

C:\Users\jb\miniconda3\envs\transformers_nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Transformers settings

In [3]:
transformers.__version__

'4.30.2'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

#### Custom paths & imports

In [5]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_src  = os.path.join(path_to_repo, 'src')
path_to_save_llm = os.path.join(path_to_repo, 'saves', 'LLM')

In [6]:
sys.path.insert(0, path_to_src)

#### Constants

In [ ]:
hub_model_name = 'falcon'

<a id="llm"></a>

# 1.  Model with 4-bit Quantization

[Table of content](#TOC)

In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hub_model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    hub_model_name, 
    quantization_config = quantization_config,
    trust_remote_code = True,
)

## 2.1 BERT model for Token Classification

[Table of content](#TOC)

Downloaded checkpoints are by default stored in HF standard cache, which is in our case `C:\Users\me\.cache\huggingface\hub`. This directory can be changed with the `cache_dir` parameter.

In [33]:
# make model creation & training deterministic
set_seed(42)

In [34]:
base_model_path = os.path.join(path_to_save_mlm, base_model_name, 'model')

try:
    model = AutoModelForTokenClassification.from_pretrained(base_model_path, label2id = label2id, id2label = id2label)
    print('Model loaded from local checkpoint.')
except:
    model = AutoModelForMaskedLM.from_pretrained(hub_model_name)
    model.save_pretrained(base_model_path)
    model = AutoModelForTokenClassification.from_pretrained(base_model_path, label2id = label2id, id2label = id2label)
    print('Model downloaded from Huggingface model hub.')

Some weights of the model checkpoint at C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\MLM\albert\model were not used when initializing AlbertForTokenClassification: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at C:\Users\jb\Desktop\NLP\perso - Transform

Model loaded from local checkpoint.


In [35]:
model.num_parameters()

11109141

In [37]:
model = model.to(device)
model = model.train()

## 2.2 Evaluation metric

[Table of content](#TOC)

During finetuning of our model, we periodically compute performance metrics for the NER task. 

The _loss_ function used for the NER task is a [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss), and directly computed by the model (see [source](https://github.com/huggingface/transformers/blob/v4.27.2/src/transformers/models/albert/modeling_albert.py#L1092)) if the expected labels are passed to the model along inputs:

In [107]:
inputs = tokenized_datasets['trn'][0]
input_ids, labels = torch.LongTensor([inputs['input_ids']]), torch.LongTensor([inputs['labels']])

In [117]:
model(input_ids, labels = labels).loss.item()

3.257025957107544

Metrics other than the training loss are computed using the [seqeval](https://github.com/chakki-works/seqeval) utility package, whose behavior is briefly presented below.

In [31]:
metric = evaluate.load("seqeval")

Let's see how this metric works:

In [32]:
tokens = tokenized_datasets['tst'][0]['tokens'][1:-1]
labels = tokenized_datasets['tst'][0]['token_categories'][1:-1]

predictions = labels.copy()
predictions[4] = "B-Condition"
# predictions[6] = "I-Condition"

print('tokens :', tokens)
print('labels :', labels)
print('prediction: ', predictions)

metric.compute(predictions = [predictions], references = [labels])

tokens : ['▁are', '▁pregnant', '▁or', '▁lac', 't', 'ating', '▁', '.']
labels : ['O', 'B-Condition', 'O', 'B-Condition', 'I-Condition', 'I-Condition', 'O', 'O']
prediction:  ['O', 'B-Condition', 'O', 'B-Condition', 'B-Condition', 'I-Condition', 'O', 'O']


{'Condition': {'precision': 0.3333333333333333,
  'recall': 0.5,
  'f1': 0.4,
  'number': 2},
 'overall_precision': 0.3333333333333333,
 'overall_recall': 0.5,
 'overall_f1': 0.4,
 'overall_accuracy': 0.875}

The accuracy, precision, recall and f1 score is computed for each category (e.g. any term appearing after a prefix of the form 'B-', 'I-', 'E-', 'S-') encountered in the batch of items to evaluate, along with overall scores.

This metric takes the lists of labels as strings, not integers, so in order to compute performance scores during finetuning, we will need to fully decode the predictions and labels before passing them to the metric. All is cast into the `compute_metrics_finegrained` function, which is passed in argument to the trainer.

## 2.3 Run finetuning on train-valid-test split

[Table of content](#TOC)

This finetuning step is meant to compute and log quality metrics as the finetuning goes. It is run on the train-valid-test split, and consequently the fintuning doesn't benefit from all the available data. To run finetuning on all data, jump to next section

In [38]:
batch_size = 6

In [42]:
args = TrainingArguments(
    # training args
    learning_rate = 2e-5,
    weight_decay = 1e-4,
    num_train_epochs = 4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    bf16 = True,
    torch_compile = False,
    optim = "adamw_torch_fused",
    # logging args
    output_dir = os.path.join(path_to_save_ner, '_checkpoints'),
    logging_dir = os.path.join(path_to_logs, final_model_name, run_name),
    evaluation_strategy = 'steps',
    save_strategy = 'no',
    logging_steps = 500,
    report_to = ['tensorboard'],
    log_level = 'warning',
)

In [43]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForTokenClassification(tokenizer),
    train_dataset = tokenized_datasets['trn'],
    eval_dataset  = tokenized_datasets['dev'],
    compute_metrics = lambda p: compute_metrics_finegrained(p, metric, class_labels.names),
)

Remarks:

- The `data_collator` is the object used to batch and pad elements of the training & evaluation datasets.
- The learning rate scheduler is `linear` by default, see also the Huggingface documentation on [schedulers](https://huggingface.co/docs/transformers/v4.27.1/en/main_classes/optimizer_schedules#transformers.SchedulerType).
- Tensor precision is by default `float32`, but other precision are available when using recent GPUs or CPU, see Huggingface post on [floating data types](https://huggingface.co/docs/transformers/v4.15.0/performance#floating-data-types).


-> _inner_training_loop
    - training_step
         loss = model(**inputs)['loss']
         loss.backward()
    - self.optimizer.step()
    - self.lr_scheduler.step()
    - model.zero_grad()

In [44]:
trainer.train()

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Condition precision,Condition recall,Condition f1,Device precision,Device recall,Device f1,Drug precision,Drug recall,Drug f1,Measurement precision,Measurement recall,Measurement f1,Observation precision,Observation recall,Observation f1,Person precision,Person recall,Person f1,Procedure precision,Procedure recall,Procedure f1,Qualifier precision,Qualifier recall,Qualifier f1,Temporal precision,Temporal recall,Temporal f1,Value precision,Value recall,Value f1
500,1.042400,0.828838,0.482200,0.526216,0.503247,0.767393,0.547292,0.724665,0.623612,0.000000,0.000000,0.000000,0.558081,0.677914,0.612188,0.530806,0.486957,0.507937,0.833333,0.039370,0.075188,0.911392,0.486486,0.634361,0.342803,0.615646,0.440389,0.319392,0.227642,0.265823,0.260223,0.255474,0.257827,0.515823,0.515823,0.515823
1000,0.743700,0.720225,0.558992,0.553696,0.556331,0.785523,0.630952,0.709369,0.667867,0.071429,0.083333,0.076923,0.638655,0.699387,0.667643,0.568889,0.556522,0.562637,0.500000,0.031496,0.059259,0.730496,0.695946,0.712803,0.431319,0.534014,0.477204,0.390805,0.276423,0.323810,0.307692,0.277372,0.291747,0.666667,0.664557,0.665610
1500,0.679500,0.683127,0.535560,0.627922,0.578075,0.792519,0.609968,0.737094,0.667532,0.062500,0.027778,0.038462,0.592771,0.754601,0.663968,0.516667,0.673913,0.584906,0.347826,0.062992,0.106667,0.785714,0.743243,0.763889,0.507289,0.591837,0.546311,0.328244,0.466125,0.385218,0.387500,0.452555,0.417508,0.618529,0.718354,0.664714
2000,0.622700,0.671939,0.553627,0.614656,0.582548,0.800748,0.575545,0.782983,0.663426,0.230769,0.250000,0.240000,0.604878,0.760736,0.673913,0.603448,0.608696,0.606061,0.458333,0.086614,0.145695,0.718750,0.777027,0.746753,0.487730,0.540816,0.512903,0.356401,0.279133,0.313070,0.435252,0.441606,0.438406,0.661677,0.699367,0.680000
2500,0.558300,0.631771,0.559226,0.620341,0.588200,0.796788,0.618789,0.761950,0.682948,0.282609,0.361111,0.317073,0.605042,0.662577,0.632504,0.584000,0.634783,0.608333,0.294118,0.039370,0.069444,0.755102,0.750000,0.752542,0.465608,0.598639,0.523810,0.368700,0.376694,0.372654,0.434211,0.481752,0.456747,0.658046,0.724684,0.689759
3000,0.571900,0.625501,0.568016,0.638345,0.601130,0.803960,0.679034,0.752390,0.713832,0.333333,0.388889,0.358974,0.663818,0.714724,0.688331,0.592308,0.669565,0.628571,0.250000,0.062992,0.100629,0.741722,0.756757,0.749164,0.447689,0.625850,0.521986,0.348387,0.439024,0.388489,0.449848,0.540146,0.490879,0.611732,0.693038,0.649852
3500,0.520300,0.654235,0.555556,0.649084,0.598689,0.799560,0.613166,0.792543,0.691410,0.388889,0.388889,0.388889,0.678571,0.757669,0.715942,0.537367,0.656522,0.590998,0.297872,0.110236,0.160920,0.763158,0.783784,0.773333,0.489971,0.581633,0.531882,0.319290,0.390244,0.351220,0.464968,0.532847,0.496599,0.631728,0.705696,0.666667
4000,0.447600,0.600694,0.583264,0.662666,0.620435,0.815314,0.657051,0.783939,0.714908,0.340909,0.416667,0.375000,0.663158,0.773006,0.713881,0.564885,0.643478,0.601626,0.243243,0.070866,0.109756,0.801471,0.736486,0.767606,0.493573,0.653061,0.562225,0.385542,0.433604,0.408163,0.480114,0.616788,0.539936,0.670659,0.708861,0.689231
4500,0.443900,0.624476,0.579545,0.676563,0.624308,0.811925,0.676710,0.794455,0.730871,0.300000,0.416667,0.348837,0.596330,0.797546,0.682415,0.635193,0.643478,0.639309,0.271605,0.173228,0.211538,0.761006,0.817568,0.788274,0.477401,0.574830,0.521605,0.363436,0.447154,0.400972,0.482940,0.671533,0.561832,0.709375,0.718354,0.713836
5000,0.436300,0.604508,0.571072,0.650979,0.608413,0.812673,0.667502,0.763862,0.712439,0.377778,0.472222,0.419753,0.644836,0.785276,0.708160,0.590551,0.652174,0.619835,0.202532,0.125984,0.155340,0.775510,0.770270,0.772881,0.454039,0.554422,0.499234,0.354839,0.447154,0.395683,0.462963,0.547445,0.501672,0.675439,0.731013,0.702128


TrainOutput(global_step=6732, training_loss=0.5380558233776761, metrics={'train_runtime': 512.1331, 'train_samples_per_second': 78.847, 'train_steps_per_second': 13.145, 'total_flos': 93310281261864.0, 'train_loss': 0.5380558233776761, 'epoch': 4.0})

In [45]:
test_results = trainer.evaluate(eval_dataset = tokenized_datasets['tst'], metric_key_prefix = 'test')

In [46]:
for k, v in test_results.items():
    print(str(k) + ' ' + '-'*(30 - len(k)) + ' {:2f}'.format(100*v))

test_loss --------------------- 62.725490
test_precision ---------------- 60.949612
test_recall ------------------- 65.299766
test_f1 ----------------------- 63.049743
test_accuracy ----------------- 82.283105
test_Condition precision ------ 69.387755
test_Condition recall --------- 76.527331
test_Condition f1 ------------- 72.782875
test_Device precision --------- 42.222222
test_Device recall ------------ 52.777778
test_Device f1 ---------------- 46.913580
test_Drug precision ----------- 70.637119
test_Drug recall -------------- 73.275862
test_Drug f1 ------------------ 71.932299
test_Measurement precision ---- 58.997050
test_Measurement recall ------- 64.308682
test_Measurement f1 ----------- 61.538462
test_Observation precision ---- 23.684211
test_Observation recall ------- 17.647059
test_Observation f1 ----------- 20.224719
test_Person precision --------- 84.768212
test_Person recall ------------ 80.503145
test_Person f1 ---------------- 82.580645
test_Procedure precision ------ 52

In [47]:
del model

torch.cuda.empty_cache()

## 2.4 Run finetuning on all data

[Table of content](#TOC)

In [48]:
# make model creation & training deterministic
set_seed(42)

In [49]:
base_model_path = os.path.join(path_to_save_mlm, base_model_name, 'model')

model = AutoModelForTokenClassification.from_pretrained(base_model_path, label2id = label2id, id2label = id2label)

Some weights of the model checkpoint at C:\Users\jb\Desktop\NLP\perso - Transformers for NLP\saves\MLM\albert\model were not used when initializing AlbertForTokenClassification: ['predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at C:\Users\jb\Desktop\NLP\perso - Transform

In [50]:
model = model.to(device)
model = model.train()

In [51]:
batch_size = 6

In [52]:
args = TrainingArguments(
    # training args
    learning_rate = 2e-5,
    weight_decay = 1e-4,
    num_train_epochs = 4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    bf16 = True,
    torch_compile = False,
    optim = "adamw_torch_fused",
    # logging args
    output_dir = os.path.join(path_to_save_ner, '_checkpoints'),
    evaluation_strategy = 'no',
    save_strategy = 'no',
)

In [53]:
metric = evaluate.load("seqeval")

In [54]:
trainer = Trainer(
    model,
    args,
    data_collator = DataCollatorForTokenClassification(tokenizer),
    train_dataset = tokenized_datasets['all'],
)

In [55]:
trainer.train()

Step,Training Loss
500,1.037500
1000,0.748900
1500,0.677200
2000,0.676600
2500,0.609400
3000,0.540400
3500,0.520700
4000,0.533400
4500,0.477900
5000,0.441100


TrainOutput(global_step=8424, training_loss=0.5213984827257176, metrics={'train_runtime': 559.7286, 'train_samples_per_second': 90.279, 'train_steps_per_second': 15.05, 'total_flos': 115325300542644.0, 'train_loss': 0.5213984827257176, 'epoch': 4.0})

In [56]:
model = model.to('cpu')

In [57]:
tokenizer.save_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'tokenizer'))
model.save_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'model'))

<a id="inference"></a>

# 3. Inference

[Table of content](#TOC)

In [58]:
tokenizer = AutoTokenizer.from_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'tokenizer'))
model = AutoModelForTokenClassification.from_pretrained(os.path.join(path_to_save_ner, final_model_name, run_name, 'model'))

In [59]:
ner = pipeline(
    task = 'ner', 
    model = model, 
    tokenizer = tokenizer,
    framework = 'pt',
    aggregation_strategy = 'simple',
)

In [60]:
sent = 'Radiologic examination of chest'


df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [61]:
sent = 'Participants with at least two of the following: serum M-protein >= 0.5 g/dL, urine M-protein >=200 mg/24 hours, an involved FLC assay >=100 mg/L, and a serum FLC ratio <0.26 or a serum FLC ratio  >1.65'


df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [62]:
sent = 'Participants with at least two of the following: Serum M-protein >= 0.5 g/dL measured using serum protein immunoelectrophoresis, Urine M-protein >=200 mg/24 hours measured using urine protein immunoelectrophoresis, Serum free light chain (FLC) assay with an involved FLC assay >=100 mg/L, and a serum FLC ratio < 0.26 or a serum FLC ratio  > 1.65'


df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [63]:
sent = 'Polyneuropathy of other causes, including but not limited to hereditary demyelinating neuropathies, neuropathies secondary to infection or systemic disease, diabetic neuropathy, drug- or toxin-induced neuropathies, multifocal motor neuropathy, monoclonal gammopathy of uncertain significance, lumbosacral radiculoplexus neuropathy, pure sensory CIDP and acquired demyelinating symmetric (DADS) neuropathy (also known as distal CIDP).'

df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [64]:
sent = 'Adults and adolescent patients with a physician diagnosis of asthma ' +\
    'for ≥12 months, based on the Global Initiative for Asthma (GINA) 2014 Guidelines ' +\
    'and the followingcriteria: A) Existing treatment with medium to high dose ICS ' +\
    '(≥250 mcg of fluticasone propionate twice daily or equipotent ICS daily dosage ' +\
    'to a maximum of 2000 mcg/day of fluticasone propionate or equivalent) in combination ' +\
    'with a second controller (eg, LABA, LTRA) for at least 3 months with a stable dose ≥1 ' +\
    'month prior to Visit 1.'

df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [65]:
sent = 'Biologics treatment: Cell-depleting agents, eg. Rituximab Drug within 6 months before baseline or until lymphocyte count returns to normal, Other biologics: within 5 half-lives or 16 weeks prior baseline.'

df_ents = parse_trf_ner_output(ner(sent))
df_ents = correct_entity_boundaries(sent, df_ents)
df_ents = remove_entity_overlaps(df_ents)
HTML(render_ner_as_html(sent, df_ents))

In [70]:
sents = '''Active hepatitis or patients with positive HBsAg, or patients with positive HBcAb plus positive HBV DNA, or positive HCV antibody (confirmed with presence of HCV RNA if needed) at screening.

History of alcohol or drug abuse within 2 years of the screening visit.

History of HIV infection or positive HIV serology at screening.

History of malignancy within 5 years before screening, except completely treated cervix carcinoma, completely treated and resolved non-metastatic squamous or basal cell carcinoma of the skin.

Infection requires systemic antibiotics, antivirals, antiparasitics, antiprotozoals, antifungals treatment within 2 weeks before baseline, superficial skin infections within 1 week before baseline visit.

Initiation AD treatment with prescription moisturizers or moisturizers containing  additives such as ceramide, hyaluronic acid, urea, or filaggrin degradation products during the screening period.

Known or suspected history of immunosuppression, including history of invasive opportunistic infections, despite infection resolution, or unusually frequent, recurrent, or prolonged infections.
'''
df_list = []
html = ''
for sent in sents.split('\n'):
    if sent.strip():
        ents = parse_trf_ner_output(ner(sent))
        ents = correct_entity_boundaries(sent, ents)
        ents = remove_entity_overlaps(ents)
        df_list.append(ents)
        html += render_ner_as_html(sent, ents)

df_ents = pd.concat(df_list, ignore_index = True)
HTML(html)

[Table of content](#TOC)